### Installing essential packages

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install PyPDF2
!pip install pywsd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wn: filename=wn-0.0.23-py3-none-any.whl size=31792911 sha256=e5e9652b660934754fe947ee38afd407859c0d293c9f2deea841c961535aba3f
  Stored in directory: /root/.cache/pip/wheels/a1/1a/7d/23a76ce45998af60e47466a694c237fa

### Importing packages

In [ ]:
import PyPDF2 as pdf
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('averaged_perceptron_tagger')
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk
import random
import requests
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from textblob import TextBlob, Word
import string
import re

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Warming up PyWSD (takes ~10 secs)... took 11.817445993423462 secs.


## Creating function for extract text from pdf for checking to create mca questions using that pdf content

In [ ]:
def get_book(path):
  pdf_reader = pdf.PdfReader(path,'rb')
  book = []

  for i in range(0,len(pdf_reader.pages)-1):
    page = pdf_reader.pages[i]
    text = page.extract_text()
    raw_txt = text.replace("\n"," ").replace("Fig."," ").replace(","," ").replace("PMRationalised 2023-24","").replace("4/22/2022","").replace("chap 1-4.indd","").replace("\uf086","")
    raw_txt = re.sub("[%s]" % re.escape(punctuation.replace(".","")), "", raw_txt)
    raw_txt = re.sub(r'\b\d{2}:\d{2}\b', '', raw_txt)
    book.append(raw_txt)

  return book

book_content =  get_book('/content/drive/MyDrive/Dataset/chapter-2.pdf')
txt = "".join(book_content)

In [ ]:
# Extracted text
txt

'From Trade to Territory                    The Company Establishes Power2 Aurangzeb was the last of the powerful Mughal rulers. He  established control over a very large part of the territory that is now known as India. After his death in 1707  many Mughal governors  subadars and big zamindars  began asserting their authority and establishing regional kingdoms. As powerful regional kingdoms emerged in various parts of India  Delhi could no longer function as an effective centre. By the second half of the eighteenth century   however  a new power was emerging on the political horizon – the British. Did you know that the British originally came as a small trading company and were reluctant to acquire territories How then did they come to be masters of a vast empire In this chapter you will see how this came about.   1 – Bahadur Shah Zafar  and his sons being arrested by Captain Hodson After Aurangzeb there was no  powerful Mughal ruler  but Mughal emperors continued to be symbolically i

In [ ]:
# Using Spacy library to get th entites are available in nlp to use NER method
nlp = spacy.load("en_core_web_sm")

entity_labels = nlp.get_pipe("ner").labels

print(entity_labels)

('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')


In [ ]:
# Getting the keywords for each tokenized sentence with some certain conditions
nlp = spacy.load("en_core_web_sm")

doc = nlp(txt)
entity_labels = ["PERSON", "ORG", "LOCATION", "DATE", "LAW", "SCIENTIFIC_TERM","QUANTITY", "GPE", "LOC","CHEMICAL"] # These are entites which is used to get keyword from text

sentences_with_multiple_entities = []
for sentence in doc.sents:
    entities = [ent.text for ent in sentence.ents]
    ent_lables = [ent.label_ for ent in sentence.ents]
    imp_entity_count = "Not greater than 2"
    for i in entity_labels:
      if ent_lables.count(i) >= 2:
        imp_entity_count = "Greater than 2"
        entity = i
        break
      else:
        continue
    tokens = [token.text.lower() for token in sentence]

    '''in each sentence extracted from text should have more tha two entities and
      the sentence should have words like and,or,such as,examples.because,these kind of sentences can have higher probability of being an option for create mca questions and also
      the sentence should have same entity twice in it.because,we can use those keywords to create a question for that sentence'''
    if len(entities) >= 2 and ("and" in tokens or "or" in tokens or "for example" in tokens or "such as" in tokens or "reasons" in tokens or "sources" in tokens or "like" in tokens) and (imp_entity_count == "Greater than 2"):
      answers = [ent.text for ent in sentence.ents if ent.label_ == entity]
      sentences_with_multiple_entities.append([answers,sentence.text])

In [ ]:
len(sentences_with_multiple_entities)  # Checking no.of.sentences satisfied the conditions

29

In [ ]:
for i in sentences_with_multiple_entities: # just to check what's the keyword and sentence for that
  print(i[0][0],":",i[1])
  break

the west coast : By the time the first English ships sailed down the west coast of Africa  round the Cape of Good Hope  and crossed the Indian Ocean  the Portuguese had already established their presence in the western coast of India  and had their base in Goa.


## Creating distractors function for those sentences to get false options using the keywords we extracted from thos sentences
**Type 1:** Wordnet --- It get the sense of the sentence using the keyword and then the distractors will be decided by the hypernyms and collect all the hyponyms which comes under that hypernym.<br>
**Type 2:** conceptnet --- If the hypernyms not founded in the wordnet another method is used to get distractors is conceptnet.<br>
**Type 3:** Some times Names can be get as a keywords by entity like "person" on that time human names are not having distractors in conceptnet or in wordnet for that case I cerated one function which is if_name.which get the word and find the antonyms and synonyms for that word and assing those as distractors.

In [ ]:
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = []
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term']

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)

    return distractor_list

def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0:
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()

    if len(word.split())>0:
        word = word.replace(" ","_")


    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return [synsets[lowest_index],synsets[lowest_index].definition()]
    else:
        return None

def if_name(word):
  # Find synonyms and antonyms
  synonyms = []
  antonyms = []
  for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
            if lemma.antonyms():
                antonyms.append(lemma.antonyms()[0].name())
  distractors = []
  if len(synonyms) + len(antonyms) >= 3:
      distractors = random.sample(synonyms + antonyms, k=3)
  else:
      distractors.append("Not enough words to generate distractors.")

  return distractors

In [ ]:
get_distractors_wordnet(get_wordsense("Srivatsan loves to watch cricket during his free time","cricket")[0],"cricket") # Checking the distractors are working or not

['Ball Game',
 'Field Hockey',
 'Football',
 'Hurling',
 'Lacrosse',
 'Polo',
 'Pushball',
 'Ultimate Frisbee']

In [ ]:
get_distractors_conceptnet("Egypt")

['Kuwait',
 'Saudi Arabia',
 'Iraq',
 'Jordan',
 'Israel',
 'Fertile Crescent',
 'Turkey',
 'Iran',
 'Lebanon',
 'Shari',
 'Mauritania',
 'Nigeria',
 'Somali peninsula',
 'Sierra Leone',
 'Malawi',
 'North Africa',
 'Senegal',
 'Mozambique',
 'Lake Tanganyika']

### Creating function to generate the mca questions using pretrained model in T5 transformer

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

def generate_mca_question(text,model):
    # Initialize T5 model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model)
    tokenizer = T5Tokenizer.from_pretrained("t5-base")

    # Create a text-to-text format for question generation
    input_text = "generate multiple answers question: " + text + " </s>"

    # Tokenize and generate the question
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    output = model.generate(input_ids)

    # Decode and print the question
    question = tokenizer.decode(output[0], skip_special_tokens=True)
    return question

#### used these two ramsrigouthamg/t5_squad_v1,mrm8488/t5-base-finetuned-question-generation-ap for to generate question these are huggingface question generators which is already trained with squad dataset that contains large amount of data (questions and answers) by those two persons.

In [ ]:
model_questions = {}

pre_trained_models = ["ramsrigouthamg/t5_squad_v1","mrm8488/t5-base-finetuned-question-generation-ap"]

for model in pre_trained_models:
  mcq_lst = {}
  for i, sentence in enumerate(sentences_with_multiple_entities):
      text_to_generate_question_from = sentence[1]
      mca_question = generate_mca_question(text_to_generate_question_from,model)
      sense = get_wordsense(text_to_generate_question_from,sentence[0][0])
      if sense is not None:
        distractors = get_distractors_wordnet(sense[0],sentence[0][0])
        if len(distractors) == 0:
          distractors = get_distractors_conceptnet(sentence[0][0])
      else:
        distractors = if_name(sentence[0][0])
      if len(distractors) == 0 or distractors is str:
        mcq_lst[mca_question]=sentence[0]
      else:
        mcq_lst[mca_question]=distractors

  model_questions[model] = [mcq_lst]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from you

### Final User Defined Function created which is used to generate MCA Questions
#####**Note:** This model can also have some false predictions.But,I checked manually mose of the questions this model generated are similar to what you expected.I tried to use gpt-2 llm model.but,it requires api key to access.I tried my best to reach this level.
####Finally,Thanks for this opportunity to explore learn somethings by this assignment.

In [ ]:
def get_mca_questions(txt:str):
  model_questions = {}
  nlp = spacy.load("en_core_web_sm")

  doc = nlp(txt)
  entity_labels = ["PERSON", "ORG", "LOCATION", "DATE", "LAW", "SCIENTIFIC_TERM","QUANTITY", "GPE", "LOC","CHEMICAL"]
    
  sentences_with_multiple_entities = []
  for sentence in doc.sents:
      entities = [ent.text for ent in sentence.ents]
      ent_lables = [ent.label_ for ent in sentence.ents]
      imp_entity_count = "Not greater than 2"
      for i in entity_labels:
        if ent_lables.count(i) >= 2:
          imp_entity_count = "Greater than 2"
          entity = i
          break
        else:
          continue
      tokens = [token.text.lower() for token in sentence]
      if len(entities) >= 2 and ("and" in tokens or "or" in tokens or "for example" in tokens or "such as" in tokens or "reasons" in tokens or "sources" in tokens or "like" in tokens) and (imp_entity_count == "Greater than 2"):
        answers = [ent.text for ent in sentence.ents if ent.label_ == entity]
        sentences_with_multiple_entities.append([answers,sentence.text])

  pre_trained_models = ["ramsrigouthamg/t5_squad_v1","mrm8488/t5-base-finetuned-question-generation-ap"]

  for model in pre_trained_models:
    mcq_lst = {}
    for i, sentence in enumerate(sentences_with_multiple_entities):
        text_to_generate_question_from = sentence[1]
        mca_question = generate_mca_question(text_to_generate_question_from,model)
        sense = get_wordsense(text_to_generate_question_from,sentence[0][0])
        if sense is not None:
          distractor = get_distractors_wordnet(sense[0],sentence[0][0])
          if len(distractor) == 0:
            distractor = get_distractors_conceptnet(sentence[0][0])
        else:
          distractor = if_name(sentence[0][0])
        if len(distractor) == 0 or type(distractor[0]) is str:
          mcq_lst[mca_question]=sentence[0]
        else:
          mcq_lst[mca_question]=distractors

    model_questions[model] = [mcq_lst]

  return model_questions

#### Tested with the content given in the pdf you given:Photosyntesis content

In [ ]:
text = '''Photosynthesis is a process used by plants and other organisms to convert light energy into
chemical energy that, through cellular respiration, can later be released to fuel the organism's
activities. Some of this chemical energy is stored in carbohydrate molecules, such as sugars and
starches, which are synthesized from carbon dioxide and water – hence the name photosynthesis,
from the Greek phōs, "light", and synthesis , "putting together". Most plants, algae, and
cyanobacteria perform photosynthesis; such organisms are called photoautotrophs. Photosynthesis
is largely responsible for producing and maintaining the oxygen content of the Earth's atmosphere,
and supplies most of the energy necessary for life on Earth.
Although photosynthesis is performed differently by different species, the process always begins
when energy from light is absorbed by proteins called reaction centers that contain green chlorophyll
(and other colored) pigments/chromophores. In plants, these proteins are held inside organelles
called chloroplasts, which are most abundant in leaf cells, while in bacteria they are embedded in
the plasma membrane. In these light-dependent reactions, some energy is used to strip electrons
from suitable substances, such as water, producing oxygen gas. The hydrogen freed by the splitting
of water is used in the creation of two further compounds that serve as short-term stores of energy,
enabling its transfer to drive other reactions: these compounds are reduced nicotinamide adenine
dinucleotide phosphate (NADPH) and adenosine triphosphate (ATP), the "energy currency" of cells.
In plants, algae and cyanobacteria, sugars are synthesized by a subsequent sequence of
light-independent reactions called the Calvin cycle. In the Calvin cycle, atmospheric carbon dioxide
is incorporated into already existing organic carbon compounds, such as ribulose bisphosphate
(RuBP).[5] Using the ATP and NADPH produced by the light-dependent reactions, the resulting
compounds are then reduced and removed to form further carbohydrates, such as glucose. In other
bacteria, different mechanisms such as the reverse Krebs cycle are used to achieve the same end.
The first photosynthetic organisms probably evolved early in the evolutionary history of life and most
likely used reducing agents such as hydrogen or hydrogen sulfide, rather than water, as sources of
electrons. Cyanobacteria appeared later; the excess oxygen they produced contributed directly to
the oxygenation of the Earth, which rendered the evolution of complex life possible. Today, the
average rate of energy capture by photosynthesis globally is approximately 130 terawatts, which is
about eight times the current power consumption of human civilization. Photosynthetic organisms
also convert around 100–115 billion tons (91–104 Pg petagrams, or billion metric tons), of carbon
into biomass per year. That plants receive some energy from light – in addition to air, soil, and water
– was first discovered in 1779 by Jan Ingenhousz.'''
question_answers = get_mca_questions(text)
question_answers

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


{'ramsrigouthamg/t5_squad_v1': [{'question: What is photosynthesis responsible for producing?': ['Earth',
    'Earth'],
   'question: How much carbon does a photosynthetic organism convert into biomass?': ['around 100–115 billion tons',
    'billion metric tons']}],
 'mrm8488/t5-base-finetuned-question-generation-ap': [{'question: What does photosynthesis produce?': ['Earth',
    'Earth'],
   'question: How many billions of tons of carbon do photosynthetic organisms convert into': ['around 100–115 billion tons',
    'billion metric tons']}]}

In [ ]:
question_answers.get("ramsrigouthamg/t5_squad_v1") # Questions generated by first pretrained model

[{'question: What is photosynthesis responsible for producing?': ['Earth',
   'Earth'],
  'question: How much carbon does a photosynthetic organism convert into biomass?': ['around 100–115 billion tons',
   'billion metric tons']}]

In [ ]:
question_answers.get("mrm8488/t5-base-finetuned-question-generation-ap") # Questions generated by Second pretrained model

[{'question: What does photosynthesis produce?': ['Earth', 'Earth'],
  'question: How many billions of tons of carbon do photosynthetic organisms convert into': ['around 100–115 billion tons',
   'billion metric tons']}]